<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE3-MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# agents/adaptive_window_agent.py
import numpy as np
import pandas as pd
import pickle
import json
import os
from collections import deque, defaultdict, Counter
from typing import Dict, List, Tuple, Optional, Any
import datetime as dt
import logging
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from statsmodels.tsa.vector_ar.var_model import VAR
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

logger = logging.getLogger(__name__)

class AdaptiveWindowAgent:
    """
    Agent A: Adaptive Window Management with Enhanced MLP

    Capabilities:
    1. Invoke new data and score using trained MLP
    2. Calculate actual performance using VAR forecast (real-time only)
    3. Track accuracy and performance statistics
    4. Monitor for drift in prediction performance
    5. Communicate with sensor agents to verify drift
    6. Retrain MLP when drift is confirmed
    """

    def __init__(self, agent_id: str = "adaptive_window_agent",
                 model_path: str = None,
                 checkpoint_path: str = None):
        self.agent_id = agent_id
        self.model_path = model_path or "/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras"
        self.checkpoint_path = checkpoint_path or "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/ckp2.weights.h5"

        # Core MLP components
        self.model = None
        self.transformer = StandardScaler()
        self.transformer_fitted = False
        self.is_model_loaded = False

        # Performance tracking - Updated to use MSE/MAE/MAPE only
        self.prediction_history = deque(maxlen=1000)
        self.mse_history = deque(maxlen=200)
        self.mape_history = deque(maxlen=200)
        self.mae_history = deque(maxlen=200)

        # Drift detection parameters - Updated thresholds
        self.drift_detection_window = 50
        self.drift_threshold_mse = 0.2    # 20% increase in MSE
        self.drift_threshold_mape = 0.2   # 20% increase in MAPE
        self.drift_threshold_mae = 0.2    # 20% increase in MAE
        self.consecutive_poor_predictions = 0
        self.drift_confirmed = False

        # Statistics storage - Updated for new metrics
        self.performance_stats = {
            'total_predictions': 0,
            'avg_mse': 0.0,
            'avg_mae': 0.0,
            'avg_mape': 0.0,
            'last_retrain_time': None,
            'drift_events': 0,
            'retraining_events': 0
        }

        # Retraining data storage
        self.retraining_data = {
            'x_buffer': deque(maxlen=10000),
            'y_buffer': deque(maxlen=10000)
        }

        # Sensor agents for drift confirmation
        self.sensor_agents = {}

        self.load_model()
        print(f"AdaptiveWindowAgent {self.agent_id} initialized")
        print(f"Model loaded: {self.is_model_loaded}")
        print(f"Transformer fitted: {self.transformer_fitted}")

    def load_model(self):
        """Load trained MLP model and recreate transformer using original training data"""
        try:
            if os.path.exists(self.model_path):
                self.model = keras.models.load_model(self.model_path)
                print(f"Loaded MLP model from {self.model_path}")
                self.is_model_loaded = True

                # Try to load saved transformer first
                transformer_path = self.model_path.replace('.keras', '_transformer.pkl')
                if os.path.exists(transformer_path):
                    with open(transformer_path, 'rb') as f:
                        self.transformer = pickle.load(f)
                    self.transformer_fitted = True
                    print("Loaded saved transformer")
                else:
                    # Recreate transformer from original training data
                    print("No saved transformer found, recreating from original training data...")

                    try:
                        # Load your original y training data
                        y_original = np.load('/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy')

                        # Fit transformer on original training data (same as your training code)
                        self.transformer = StandardScaler()
                        self.transformer.fit(y_original.reshape(-1, 1))
                        self.transformer_fitted = True

                        # Save it for future use
                        with open(transformer_path, 'wb') as f:
                            pickle.dump(self.transformer, f)

                        print(f"Fitted transformer on {len(y_original)} original training samples and saved")

                    except Exception as e:
                        print(f"Could not load original training data: {e}")
                        self.transformer = StandardScaler()
                        self.transformer_fitted = False

            else:
                print(f"Model file not found at {self.model_path}")
                self.is_model_loaded = False

        except Exception as e:
            print(f"Error loading model: {e}")
            self.is_model_loaded = False

    def evaluate_forecast_performance(self, sequence_3d: np.ndarray, predicted_window: int, n_future: int = 1) -> Dict[str, float]:
        """
        Use predicted window to forecast with VAR and calculate MSE, MAE, MAPE
        Handles constant columns and other VAR issues robustly
        """
        try:
            # Your existing VAR forecast code
            x1 = sequence_3d  # Get the subsequences feature here
            df = pd.DataFrame(x1, columns=['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12'])
            df_train, df_test = df[0:-n_future], df[-n_future:]

            # Check for constant columns and remove them
            constant_columns = []
            df_train_cleaned = df_train.copy()

            # More robust constant column detection
            for col in df_train_cleaned.columns:
                col_data = df_train_cleaned[col]
                # Check for constant values (all same) or near-constant (very low variance)
                if col_data.nunique() <= 1 or col_data.var() < 1e-12:
                    constant_columns.append(col)

            if constant_columns:
                print(f"Removing constant columns: {constant_columns}")
                df_train_cleaned = df_train_cleaned.drop(columns=constant_columns)
                df_test_cleaned = df_test.drop(columns=constant_columns)
            else:
                df_test_cleaned = df_test

            # Double-check: verify no constant columns remain
            remaining_constant = []
            for col in df_train_cleaned.columns:
                if df_train_cleaned[col].nunique() <= 1 or df_train_cleaned[col].var() < 1e-12:
                    remaining_constant.append(col)

            if remaining_constant:
                print(f"Still found constant columns after cleaning: {remaining_constant}")
                df_train_cleaned = df_train_cleaned.drop(columns=remaining_constant)
                df_test_cleaned = df_test_cleaned.drop(columns=remaining_constant)
                constant_columns.extend(remaining_constant)

            # Check if we have enough non-constant columns for VAR
            if len(df_train_cleaned.columns) < 2:
                print(f"Not enough non-constant columns ({len(df_train_cleaned.columns)}) for VAR modeling")
                return {
                    'mse': 99999,
                    'mape': 99999,
                    'mae': 99999,
                    'forecast_success': False
                }

            # Check if predicted window is valid (not larger than available data)
            k = min(predicted_window, len(df_train_cleaned) - 2)  # Leave room for VAR lag requirements
            if k < 1:
                k = 1

            # Try VAR modeling with cleaned data
            model = VAR(df_train_cleaned)

            # Try fitting with different trend options if default fails
            model_fitted = None
            trend_options = ['n', 'c', 'ct', 'ctt']  # Start with 'n' (no trend) first

            for trend in trend_options:
                try:
                    model_fitted = model.fit(maxlags=k, trend=trend)
                    print(f"VAR fit successful with trend='{trend}', lags={k}")
                    break
                except Exception as trend_e:
                    print(f"VAR fit failed with trend='{trend}': {trend_e}")
                    continue

            if model_fitted is None:
                print(f"VAR could not fit with any trend option")
                return {
                    'mse': 99999,
                    'mape': 99999,
                    'mae': 99999,
                    'forecast_success': False
                }

            # Make forecast
            forecast_input = df_train_cleaned.values[-model_fitted.k_ar:]
            fc = model_fitted.forecast(y=forecast_input, steps=n_future)
            df_forecast = pd.DataFrame(fc, index=df.index[-n_future:], columns=df_train_cleaned.columns)

            # Calculate metrics - use V1 if available, otherwise use first available column
            target_col = 'V1' if 'V1' in df_forecast.columns else df_forecast.columns[0]

            if target_col not in df_test_cleaned.columns:
                print(f"Target column {target_col} was removed due to being constant")
                return {
                    'mse': 99999,
                    'mape': 99999,
                    'mae': 99999,
                    'forecast_success': False
                }

            # Calculate error metrics
            actual = df_test_cleaned[target_col].values[0]
            predicted = df_forecast[target_col].values[0]

            # Avoid division by zero in relative metrics
            if abs(actual) < 1e-10:
                print(f"Actual value too close to zero ({actual}) for MAPE calculation")
                mape = 99999
            else:
                mape = abs((actual - predicted) / actual)

            # Calculate MSE and MAE
            mse = ((actual - predicted) ** 2) / max(abs(actual), 1e-10)
            mae = abs(actual - predicted)

            return {
                'mse': mse,
                'mape': mape,
                'mae': mae,
                'forecast_success': True,
                'target_column': target_col,
                'constant_columns_removed': constant_columns,
                'var_lags_used': model_fitted.k_ar
            }

        except Exception as e:
            print(f'VAR could not solve for window {predicted_window}: {e}')
            return {
                'mse': 99999,
                'mape': 99999,
                'mae': 99999,
                'forecast_success': False,
                'error_details': str(e)
            }

    def _get_most_frequent_window(self) -> int:
        """Get the most frequently occurring window size from original training data"""
        try:
            # Load your original training ground truth data
            y_training_data = np.load('/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy')

            # Find most frequent window size from training data
            window_counts = Counter(y_training_data.astype(int))
            most_frequent_window = window_counts.most_common(1)[0][0]

            print(f"Using most frequent window from training data: {most_frequent_window}")
            return most_frequent_window

        except Exception as e:
            print(f"Could not load training data for fallback: {e}")
            return 20  # Only if training data unavailable

    def predict_window_size(self, feature_vector: np.ndarray, sequence_3d: np.ndarray) -> Dict[str, Any]:
        """
        Predict window size using MLP and calculate VAR performance in real-time
        """
        if not self.is_model_loaded:
            return {
                'predicted_window': 20,
                'confidence': 0.0,
                'error': "Model not loaded"
            }

        try:
            # Ensure feature vector is 2D
            if feature_vector.ndim == 1:
                feature_vector = feature_vector.reshape(1, -1)

            print(f"Input feature vector shape: {feature_vector.shape}")

            # 1. INVOKE NEW DATA AND SCORE USING MLP
            try:
                prediction_raw = self.model.predict(feature_vector, verbose=0)
            except Exception as e:
                if "input shape" in str(e) and "32" in str(e):
                    print("Padding input to batch size 32")
                    feature_batch = np.repeat(feature_vector, 32, axis=0)
                    prediction_batch = self.model.predict(feature_batch, verbose=0)
                    prediction_raw = prediction_batch[0:1]
                else:
                    raise e

            print(f"Raw prediction: {prediction_raw}")

            # 2. TRANSFORM PREDICTION BACK TO ORIGINAL SCALE
            if self.transformer_fitted:
                predicted_window = self.transformer.inverse_transform(prediction_raw)[0, 0]
                print(f"After inverse transform: {predicted_window}")
            else:
                predicted_window = prediction_raw[0, 0]
                logger.warning("Transformer not fitted yet, using raw prediction")
                print(f"Using raw prediction: {predicted_window}")

            predicted_window = int(round(predicted_window))
            print(f"Final predicted window: {predicted_window}")

            # Create prediction record
            prediction_record = {
                'timestamp': dt.datetime.now(),
                'predicted_window': predicted_window,
                'feature_vector': feature_vector.flatten(),
                'raw_prediction': prediction_raw[0, 0],
                'transformer_fitted': self.transformer_fitted
            }

            # 3. CALCULATE ACTUAL PERFORMANCE AGAINST VAR FORECAST
            forecast_metrics = self.evaluate_forecast_performance(
                sequence_3d, predicted_window, n_future=1
            )

            prediction_record.update({
                'forecast_metrics': forecast_metrics,
                'mse': forecast_metrics.get('mse', 99999),
                'mape': forecast_metrics.get('mape', 99999),
                'mae': forecast_metrics.get('mae', 99999),
                'forecast_success': forecast_metrics.get('forecast_success', False)
            })

            # 4. TRACK PERFORMANCE STATISTICS (Updated to use MSE/MAE/MAPE)
            if forecast_metrics.get('forecast_success', False):
                # Store individual metrics
                self.mse_history.append(forecast_metrics['mse'])
                self.mape_history.append(forecast_metrics['mape'])
                self.mae_history.append(forecast_metrics['mae'])

                # Update performance statistics
                self.performance_stats.update({
                    'total_predictions': self.performance_stats['total_predictions'] + 1,
                    'avg_mse': np.mean(self.mse_history),
                    'avg_mape': np.mean(self.mape_history),
                    'avg_mae': np.mean(self.mae_history)
                })

                # Add current metrics to prediction record
                prediction_record.update({
                    'current_mse': forecast_metrics['mse'],
                    'current_mape': forecast_metrics['mape'],
                    'current_mae': forecast_metrics['mae'],
                    'recent_avg_mse': np.mean(list(self.mse_history)[-10:]) if len(self.mse_history) >= 10 else forecast_metrics['mse'],
                    'recent_avg_mape': np.mean(list(self.mape_history)[-10:]) if len(self.mape_history) >= 10 else forecast_metrics['mape'],
                    'recent_avg_mae': np.mean(list(self.mae_history)[-10:]) if len(self.mae_history) >= 10 else forecast_metrics['mae']
                })

                # 5. CHECK FOR DRIFT (Updated to use new metrics)
                drift_detected = self._check_for_drift()
                prediction_record['drift_detected'] = drift_detected

                if drift_detected:
                    prediction_record['drift_action'] = self._handle_drift_detection(feature_vector, predicted_window)
            else:
                # VAR forecast failed - treat as poor prediction
                self.consecutive_poor_predictions += 1
                prediction_record.update({
                    'drift_detected': False,
                    'forecast_failed': True
                })

            # Store prediction
            self.prediction_history.append(prediction_record)

            # Add to retraining buffer (store features and predicted window)
            self.retraining_data['x_buffer'].append(feature_vector.flatten())
            self.retraining_data['y_buffer'].append(predicted_window)

            return {
                'predicted_window': predicted_window,
                'forecast_metrics': forecast_metrics,
                'confidence': self._calculate_confidence(prediction_record),
                'performance_stats': self.get_recent_performance(),
                'drift_detected': prediction_record.get('drift_detected', False),
                'prediction_id': len(self.prediction_history),
                'transformer_status': 'fitted' if self.transformer_fitted else 'not_fitted'
            }

        except Exception as e:
            logger.error(f"Prediction error: {e}")
            return {
                'predicted_window': 20,
                'confidence': 0.0,
                'error': str(e)
            }

    def _check_for_drift(self) -> bool:
        """Monitor and identify drift in prediction performance using MSE/MAE/MAPE"""
        if len(self.mse_history) < self.drift_detection_window:
            return False

        try:
            # Get recent and historical performance for each metric
            recent_window = 20
            historical_start = self.drift_detection_window

            # MSE drift detection
            recent_mse = np.mean(list(self.mse_history)[-recent_window:])
            historical_mse = np.mean(list(self.mse_history)[-historical_start:-recent_window])
            mse_increase_ratio = recent_mse / max(historical_mse, 0.001)
            mse_drift = mse_increase_ratio > (1 + self.drift_threshold_mse)

            # MAPE drift detection
            recent_mape = np.mean(list(self.mape_history)[-recent_window:])
            historical_mape = np.mean(list(self.mape_history)[-historical_start:-recent_window])
            mape_increase_ratio = recent_mape / max(historical_mape, 0.001)
            mape_drift = mape_increase_ratio > (1 + self.drift_threshold_mape)

            # MAE drift detection
            recent_mae = np.mean(list(self.mae_history)[-recent_window:])
            historical_mae = np.mean(list(self.mae_history)[-historical_start:-recent_window])
            mae_increase_ratio = recent_mae / max(historical_mae, 0.001)
            mae_drift = mae_increase_ratio > (1 + self.drift_threshold_mae)

            # Track consecutive poor predictions (high MSE/MAE/MAPE)
            recent_high_mse = recent_mse > np.percentile(list(self.mse_history), 75)
            recent_high_mape = recent_mape > np.percentile(list(self.mape_history), 75)
            recent_high_mae = recent_mae > np.percentile(list(self.mae_history), 75)

            if recent_high_mse and recent_high_mape and recent_high_mae:
                self.consecutive_poor_predictions += 1
            else:
                self.consecutive_poor_predictions = 0

            consecutive_drift = self.consecutive_poor_predictions > 10

            # Drift detected if multiple conditions met
            drift_score = sum([mse_drift, mape_drift, mae_drift])
            drift_detected = (drift_score >= 2) or consecutive_drift

            if drift_detected:
                logger.warning(f"Drift detected: MSE ratio={mse_increase_ratio:.3f}, "
                              f"MAPE ratio={mape_increase_ratio:.3f}, MAE ratio={mae_increase_ratio:.3f}, "
                              f"Consecutive poor predictions={self.consecutive_poor_predictions}")
                self.performance_stats['drift_events'] += 1

            return drift_detected

        except Exception as e:
            logger.error(f"Drift detection error: {e}")
            return False

    def _handle_drift_detection(self, current_features: np.ndarray, predicted_window: int) -> str:
        """Handle drift detection using new metrics"""
        if self.drift_confirmed:
            return "Already handling drift"

        # Query sensor agents for their drift status
        sensor_drift_confirmations = self._query_sensor_agents_for_drift()

        # If majority of sensors also detect drift, confirm and retrain
        if sensor_drift_confirmations >= len(self.sensor_agents) * 0.6 or len(self.sensor_agents) == 0:
            self.drift_confirmed = True
            logger.info("Drift confirmed by sensor agents. Initiating retraining...")

            # Retrain MLP
            retrain_success = self._retrain_model()

            if retrain_success:
                self.drift_confirmed = False
                self.consecutive_poor_predictions = 0
                self.performance_stats['retraining_events'] += 1
                self.performance_stats['last_retrain_time'] = dt.datetime.now()
                return "Retraining completed successfully"
            else:
                return "Retraining failed"
        else:
            return f"Drift suspected but not confirmed by sensors ({sensor_drift_confirmations}/{len(self.sensor_agents)})"

    def _query_sensor_agents_for_drift(self) -> int:
        """Query sensor agents to confirm drift"""
        confirmations = 0

        for sensor_id, sensor_agent in self.sensor_agents.items():
            try:
                # This would be actual message passing in full implementation
                sensor_drift = np.random.random() > 0.7  # Simulate sensor response
                if sensor_drift:
                    confirmations += 1
            except Exception as e:
                logger.error(f"Error querying sensor {sensor_id}: {e}")

        return confirmations

    def _retrain_model(self) -> bool:
        """Retrain MLP using buffered data"""
        try:
            logger.info("Starting MLP retraining...")

            if len(self.retraining_data['x_buffer']) < 100:
                logger.warning("Insufficient data for retraining")
                return False

            # Prepare retraining data
            X_retrain = np.array(list(self.retraining_data['x_buffer']))
            y_raw = np.array(list(self.retraining_data['y_buffer']))

            # Transform y data for training if transformer is fitted
            if self.transformer_fitted:
                y_retrain = self.transformer.transform(y_raw.reshape(-1, 1)).flatten()
            else:
                y_retrain = y_raw

            # Create new model with same architecture
            new_model = Sequential()
            new_model.add(Dense(64, activation='relu', input_shape=(X_retrain.shape[1],)))
            new_model.add(Dense(32, activation='relu'))
            new_model.add(Dense(16, activation='relu'))
            new_model.add(Dense(8, activation='relu'))
            new_model.add(Dense(1))

            optimizer = keras.optimizers.Adam(learning_rate=0.0003, clipnorm=1)
            new_model.compile(loss='mean_squared_error', optimizer=optimizer,
                            metrics=['mean_squared_error'])

            es = keras.callbacks.EarlyStopping(
                patience=10, verbose=0, min_delta=0.0001,
                monitor='loss', mode='min', restore_best_weights=True
            )

            # Train the new model
            history = new_model.fit(
                X_retrain, y_retrain,
                epochs=50,
                batch_size=32,
                validation_split=0.2,
                callbacks=[es],
                verbose=0
            )

            # Evaluate new model performance
            val_loss = min(history.history['val_loss'])

            # Only replace model if new one is better
            current_recent_mse = np.mean(list(self.mse_history)[-10:]) if self.mse_history else float('inf')
            if val_loss < current_recent_mse * 1.1:
                # Replace the model
                self.model = new_model

                # Save the retrained model
                retrain_path = self.model_path.replace('.keras', '_retrained.keras')
                self.model.save(retrain_path)

                # Clear history to start fresh
                self.mse_history.clear()
                self.mape_history.clear()
                self.mae_history.clear()

                logger.info(f"Model successfully retrained. New validation loss: {val_loss:.4f}")
                return True
            else:
                logger.warning(f"New model performance worse ({val_loss:.4f} vs {current_recent_mse:.4f})")
                return False

        except Exception as e:
            logger.error(f"Retraining failed: {e}")
            return False

    def _calculate_confidence(self, prediction_record: Dict) -> float:
        """Calculate confidence based on recent MSE/MAE/MAPE performance"""
        if len(self.mse_history) < 10:
            return 0.5

        # Calculate confidence based on recent performance metrics
        recent_mse = np.mean(list(self.mse_history)[-10:])
        recent_mape = np.mean(list(self.mape_history)[-10:])
        recent_mae = np.mean(list(self.mae_history)[-10:])

        # Normalize metrics to 0-1 scale (lower is better for all three)
        # Use historical percentiles for normalization
        mse_percentile = np.percentile(list(self.mse_history), 25) if len(self.mse_history) > 20 else recent_mse
        mape_percentile = np.percentile(list(self.mape_history), 25) if len(self.mape_history) > 20 else recent_mape
        mae_percentile = np.percentile(list(self.mae_history), 25) if len(self.mae_history) > 20 else recent_mae

        # Calculate confidence scores (higher when metrics are lower)
        mse_confidence = max(0, 1 - (recent_mse / max(mse_percentile * 4, 0.001)))
        mape_confidence = max(0, 1 - (recent_mape / max(mape_percentile * 4, 0.001)))
        mae_confidence = max(0, 1 - (recent_mae / max(mae_percentile * 4, 0.001)))

        # Average confidence across metrics
        confidence = (mse_confidence + mape_confidence + mae_confidence) / 3

        # Apply penalty for forecast failures
        if not prediction_record.get('forecast_success', True):
            confidence *= 0.5

        return min(1.0, max(0.1, confidence))

    def get_recent_performance(self) -> Dict[str, Any]:
        """Get recent performance statistics using MSE/MAE/MAPE"""
        if not self.prediction_history:
            return {}

        # Get successful forecasts only
        successful_predictions = [p for p in list(self.prediction_history)[-50:]
                                if p.get('forecast_success', False)]

        performance_data = {
            'total_predictions': len(self.prediction_history),
            'successful_predictions': len(successful_predictions),
            'success_rate': len(successful_predictions) / max(len(self.prediction_history), 1),
            'drift_events': self.performance_stats['drift_events'],
            'retraining_events': self.performance_stats['retraining_events'],
            'last_retrain': self.performance_stats['last_retrain_time'],
            'consecutive_poor': self.consecutive_poor_predictions,
            'transformer_fitted': self.transformer_fitted
        }

        # Add metric statistics if available
        if self.mse_history:
            performance_data.update({
                'recent_mse': np.mean(list(self.mse_history)[-10:]),
                'avg_mse': self.performance_stats.get('avg_mse', 0.0),
                'mse_trend': self._calculate_trend(list(self.mse_history)[-20:]) if len(self.mse_history) >= 20 else 0.0
            })

        if self.mape_history:
            performance_data.update({
                'recent_mape': np.mean(list(self.mape_history)[-10:]),
                'avg_mape': self.performance_stats.get('avg_mape', 0.0),
                'mape_trend': self._calculate_trend(list(self.mape_history)[-20:]) if len(self.mape_history) >= 20 else 0.0
            })

        if self.mae_history:
            performance_data.update({
                'recent_mae': np.mean(list(self.mae_history)[-10:]),
                'avg_mae': self.performance_stats.get('avg_mae', 0.0),
                'mae_trend': self._calculate_trend(list(self.mae_history)[-20:]) if len(self.mae_history) >= 20 else 0.0
            })

        return performance_data

    def _calculate_trend(self, values: List[float]) -> float:
        """Calculate trend in performance metrics (positive = improving, negative = degrading)"""
        if len(values) < 5:
            return 0.0

        try:
            x = np.arange(len(values))
            coeffs = np.polyfit(x, values, 1)
            # Return negative slope because for MSE/MAE/MAPE, decreasing is better
            return -coeffs[0]
        except:
            return 0.0

    def connect_sensor_agents(self, sensor_agents: Dict):
        """Connect to sensor agents for drift confirmation"""
        self.sensor_agents = sensor_agents
        logger.info(f"Connected to {len(sensor_agents)} sensor agents")

    def get_performance_plot_data(self) -> Dict[str, List]:
        """Get data for performance visualization"""
        if not self.prediction_history:
            return {}

        recent_records = [p for p in self.prediction_history if p.get('forecast_success', False)]

        return {
            'timestamps': [r['timestamp'] for r in recent_records],
            'predicted': [r['predicted_window'] for r in recent_records],
            'mse_values': [r.get('mse', 0) for r in recent_records],
            'mae_values': [r.get('mae', 0) for r in recent_records],
            'mape_values': [r.get('mape', 0) for r in recent_records]
        }

    def save_performance_state(self, filepath: str):
        """Save current performance state"""
        state = {
            'performance_stats': self.performance_stats.copy(),
            'prediction_history': list(self.prediction_history)[-100:],
            'mse_history': list(self.mse_history),
            'mape_history': list(self.mape_history),
            'mae_history': list(self.mae_history),
            'transformer_fitted': self.transformer_fitted
        }

        try:
            # Convert datetime objects to strings for JSON serialization
            for record in state['prediction_history']:
                if 'timestamp' in record:
                    # Check if timestamp is already a string
                    if hasattr(record['timestamp'], 'isoformat'):
                        record['timestamp'] = record['timestamp'].isoformat()
                # Convert numpy arrays to lists for JSON serialization
                if 'feature_vector' in record and hasattr(record['feature_vector'], 'tolist'):
                    record['feature_vector'] = record['feature_vector'].tolist()

            # Handle last_retrain_time
            if state['performance_stats']['last_retrain_time']:
                if hasattr(state['performance_stats']['last_retrain_time'], 'isoformat'):
                    state['performance_stats']['last_retrain_time'] = state['performance_stats']['last_retrain_time'].isoformat()

            # Save to file
            with open(filepath, 'w') as f:
                json.dump(state, f, default=str, indent=2)

            logger.info(f"Performance state saved to {filepath}")

        except Exception as e:
            logger.error(f"Failed to save performance state: {e}")
            # Try simpler save without complex objects
            simple_state = {
                'total_predictions': len(self.prediction_history),
                'avg_mse': np.mean(self.mse_history) if self.mse_history else 0.0,
                'avg_mape': np.mean(self.mape_history) if self.mape_history else 0.0,
                'avg_mae': np.mean(self.mae_history) if self.mae_history else 0.0,
                'drift_events': self.performance_stats['drift_events'],
                'retraining_events': self.performance_stats['retraining_events']
            }

            with open(filepath.replace('.json', '_simple.json'), 'w') as f:
                json.dump(simple_state, f, indent=2)
            print(f"Saved simplified performance state to {filepath.replace('.json', '_simple.json')}")


# Test with YOUR actual data - Real-time VAR calculation only
if __name__ == "__main__":
    # Initialize the agent with your actual model
    agent = AdaptiveWindowAgent(
        model_path="/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras"
    )

    print("Loading your actual dataset...")

    # Load your actual saved dataset
    Long_train = np.load('/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
    print(f"Loaded Long_train shape: {Long_train.shape}")

    # Take last 10 entries as test module for debugging
    test_sequences = Long_train[-100:]
    print(f"Testing with last 10 sequences: {test_sequences.shape}")

    print("\nStarting real-time VAR testing...")
    print("=" * 70)

    for i in range(len(test_sequences)):
        sequence_3d = test_sequences[i]  # Shape: (50, 12)

        # Simply flatten the sequence to get 600 features (50 * 12 = 600)
        features = sequence_3d.flatten()  # This gives you exactly 600 features

        print(f"\nSample {i+1}: Using flattened sequence of size {len(features)}")

        # Test the agent with real-time VAR calculation
        result = agent.predict_window_size(features, sequence_3d=sequence_3d)

        # Handle potential missing keys safely
        mlp_pred = result.get('predicted_window', 0)
        forecast_metrics = result.get('forecast_metrics', {})
        error_msg = result.get('error', None)

        if error_msg:
            print(f"Sample {i+1}: ERROR - {error_msg}")
            continue

        # Print results
        performance = agent.get_recent_performance()

        if forecast_metrics.get('forecast_success', False):
            mse_val = forecast_metrics['mse']
            mae_val = forecast_metrics['mae']
            mape_val = forecast_metrics['mape']

            print(f"Sample {i+1:3d}: MLP={mlp_pred:2d}, "
                  f"MSE={mse_val:6.4f}, MAE={mae_val:6.4f}, MAPE={mape_val:6.4f}, "
                  f"Avg_MSE={performance.get('recent_mse', 0):6.4f}")
        else:
            print(f"Sample {i+1:3d}: MLP={mlp_pred:2d}, VAR forecast failed")

        # Check for drift detection
        if result.get('drift_detected', False):
            print(f"*** DRIFT DETECTED at sample {i+1} ***")
            drift_action = result.get('drift_action')
            if drift_action:
                print(f"Drift action: {drift_action}")

    print("\n" + "=" * 70)
    print("FINAL PERFORMANCE SUMMARY")
    print("=" * 70)

    final_performance = agent.get_recent_performance()

    print(f"Total predictions: {final_performance.get('total_predictions', 0)}")
    print(f"Successful predictions: {final_performance.get('successful_predictions', 0)}")
    print(f"Success rate: {final_performance.get('success_rate', 0):.2%}")
    print(f"Average MSE: {final_performance.get('recent_mse', 0):.4f}")
    print(f"Average MAE: {final_performance.get('recent_mae', 0):.4f}")
    print(f"Average MAPE: {final_performance.get('recent_mape', 0):.4f}")
    print(f"Drift events: {final_performance.get('drift_events', 0)}")
    print(f"Retraining events: {agent.performance_stats['retraining_events']}")
    print(f"Transformer fitted: {final_performance.get('transformer_fitted', False)}")

    # Show performance trends if enough data
    if len(agent.mse_history) >= 20:
        print(f"MSE trend: {final_performance.get('mse_trend', 0):.4f} (positive = improving)")
        print(f"MAPE trend: {final_performance.get('mape_trend', 0):.4f} (positive = improving)")
        print(f"MAE trend: {final_performance.get('mae_trend', 0):.4f} (positive = improving)")

    # Save test results
    agent.save_performance_state("real_data_test_results.json")
    print(f"\nTest results saved to: real_data_test_results.json")

Loaded MLP model from /content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras
Loaded saved transformer
AdaptiveWindowAgent adaptive_window_agent initialized
Model loaded: True
Transformer fitted: True
Loading your actual dataset...
Loaded Long_train shape: (3627, 50, 12)
Testing with last 10 sequences: (100, 50, 12)

Starting real-time VAR testing...

Sample 1: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[0.07456951]]
After inverse transform: 2.0745694637298584
Final predicted window: 2
VAR fit successful with trend='n', lags=2
Sample   1: MLP= 2, MSE=0.0310, MAE=0.1604, MAPE=0.1933, Avg_MSE=0.0310

Sample 2: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[-0.19865508]]
After inverse transform: 1.801344871520996
Final predicted window: 2
VAR fit successful with trend='n', lags=2
Sample   2: MLP= 2, MSE=0.0005, MAE=0.0206, MAPE=0.0248, Avg_MSE=0.0158

Sample 3: Using flat

Raw prediction: [[-0.00441009]]
After inverse transform: 1.9955899715423584
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  82: MLP= 2, MSE=0.0264, MAE=0.1366, MAPE=0.1931, Avg_MSE=0.0210

Sample 83: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[0.04946238]]
After inverse transform: 2.04946231842041
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  83: MLP= 2, MSE=0.0341, MAE=0.1694, MAPE=0.2014, Avg_MSE=0.0237
*** DRIFT DETECTED at sample 83 ***

Sample 84: Using flattened sequence of size 600
Input feature vector shape: (1, 600)


Raw prediction: [[0.10648827]]
After inverse transform: 2.1064882278442383
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  84: MLP= 2, MSE=0.0027, MAE=0.0434, MAPE=0.0614, Avg_MSE=0.0117
*** DRIFT DETECTED at sample 84 ***

Sample 85: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[0.10648827]]
After inverse transform: 2.1064882278442383
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  85: MLP= 2, MSE=0.0053, MAE=0.0635, MAPE=0.0829, Avg_MSE=0.0100
*** DRIFT DETECTED at sample 85 ***

Sample 86: Using flattened sequence of size 600
Input feature vector shape: (1, 600)


Raw prediction: [[0.10648827]]
After inverse transform: 2.1064882278442383
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  86: MLP= 2, MSE=0.0083, MAE=0.0802, MAPE=0.1035, Avg_MSE=0.0100
*** DRIFT DETECTED at sample 86 ***

Sample 87: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[0.02493097]]
After inverse transform: 2.024930953979492
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  87: MLP= 2, MSE=0.0000, MAE=0.0014, MAPE=0.0019, Avg_MSE=0.0098
*** DRIFT DETECTED at sample 87 ***

Sample 88: Using flattened sequence of size 600
Input feature vector shape: (1, 600)


Raw prediction: [[0.02494869]]
After inverse transform: 2.0249485969543457
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  88: MLP= 2, MSE=0.0000, MAE=0.0014, MAPE=0.0020, Avg_MSE=0.0090
*** DRIFT DETECTED at sample 88 ***

Sample 89: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[-0.07065196]]
After inverse transform: 1.9293479919433594
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  89: MLP= 2, MSE=0.0001, MAE=0.0105, MAPE=0.0136, Avg_MSE=0.0088
*** DRIFT DETECTED at sample 89 ***

Sample 90: Using flattened sequence of size 600
Input feature vector shape: (1, 600)


Raw prediction: [[-0.07603101]]
After inverse transform: 1.923969030380249
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  90: MLP= 2, MSE=0.0021, MAE=0.0376, MAPE=0.0552, Avg_MSE=0.0090
*** DRIFT DETECTED at sample 90 ***

Sample 91: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[-0.07623447]]
After inverse transform: 1.9237655401229858
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  91: MLP= 2, MSE=0.0073, MAE=0.0742, MAPE=0.0979, Avg_MSE=0.0086

Sample 92: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[-0.03084563]]
After inverse transform: 1.9691543579101562
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  92: MLP= 2, MSE=0.0066, MAE=0.0698, MAPE=0.0947, Avg_MSE=0.0066
*** DRIFT DETECTED at sample 92 ***

Sam

Raw prediction: [[-0.16598846]]
After inverse transform: 1.8340115547180176
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  93: MLP= 2, MSE=0.0495, MAE=0.2088, MAPE=0.2371, Avg_MSE=0.0082
*** DRIFT DETECTED at sample 93 ***

Sample 94: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[0.37979573]]
After inverse transform: 2.379795789718628
Final predicted window: 2
Removing constant columns: ['V12']
VAR fit successful with trend='n', lags=2
Sample  94: MLP= 2, MSE=4.1402, MAE=0.8705, MAPE=4.7559, Avg_MSE=0.4219
*** DRIFT DETECTED at sample 94 ***

Sample 95: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[-0.83759195]]
After inverse transform: 1.1624081134796143
Final predicted window: 1
VAR fit successful with trend='n', lags=1
Sample  95: MLP= 1, MSE=20.0850, MAE=1.9174, MAPE=10.4750, Avg_MSE=2.4299
*** DRIFT DETECTED at sample 95 ***

S

Raw prediction: [[-0.44911188]]
After inverse transform: 1.5508880615234375
Final predicted window: 2
VAR fit successful with trend='n', lags=2
Sample  96: MLP= 2, MSE=36.8793, MAE=2.5982, MAPE=14.1941, Avg_MSE=6.1170
*** DRIFT DETECTED at sample 96 ***

Sample 97: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[-0.02855344]]
After inverse transform: 1.9714465141296387
Final predicted window: 2
VAR fit successful with trend='n', lags=2
Sample  97: MLP= 2, MSE=0.0022, MAE=0.0201, MAPE=0.1097, Avg_MSE=6.1172
*** DRIFT DETECTED at sample 97 ***

Sample 98: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[0.9754352]]
After inverse transform: 2.975435256958008
Final predicted window: 3
VAR fit successful with trend='n', lags=3
Sample  98: MLP= 3, MSE=1.2891, MAE=1.0364, MAPE=1.2438, Avg_MSE=6.2461
*** DRIFT DETECTED at sample 98 ***

Sample 99: Using flattened sequence of size 600
Input feature vector sha

Raw prediction: [[-0.4948423]]
After inverse transform: 1.505157709121704
Final predicted window: 2
VAR fit successful with trend='n', lags=2
Sample  99: MLP= 2, MSE=1.2199, MAE=1.0083, MAPE=1.2100, Avg_MSE=6.3681
*** DRIFT DETECTED at sample 99 ***

Sample 100: Using flattened sequence of size 600
Input feature vector shape: (1, 600)
Raw prediction: [[2.174047]]
After inverse transform: 4.174046993255615
Final predicted window: 4
VAR fit successful with trend='n', lags=4
Sample 100: MLP= 4, MSE=0.3684, MAE=0.5665, MAPE=0.6504, Avg_MSE=6.4047
*** DRIFT DETECTED at sample 100 ***

FINAL PERFORMANCE SUMMARY
Total predictions: 100
Successful predictions: 50
Success rate: 50.00%
Average MSE: 6.4047
Average MAE: 0.8370
Average MAPE: 3.3068
Drift events: 17
Retraining events: 0
Transformer fitted: True
MSE trend: -0.4973 (positive = improving)
MAPE trend: -0.2458 (positive = improving)
MAE trend: -0.0664 (positive = improving)

Test results saved to: real_data_test_results.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')